In [2]:
import taichi as ti
import torch
import src_ti.simulation as simulation
import importlib
importlib.reload(simulation)
ti.init()

flow_kernel = ti.Vector.field(2, dtype=ti.i8, shape=5)
flow_kernel[0] = [0, 0]  # ORIGIN
flow_kernel[1] = [-1, 0] # UP
flow_kernel[2] = [0, 1]  # RIGHT
flow_kernel[3] = [1, 0]  # DOWN
flow_kernel[4] = [0, -1] # LEFT

num_com = 3


[Taichi] version 1.7.0, llvm 15.0.7, commit 6b563a2a, osx, python 3.10.8


[I 10/23/23 19:00:42.646 24631] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=arm64


In [3]:
import torch.nn as nn
w,h, = 100,100
class NCA():
    def __init__(self):
        self.conv = nn.Conv2d(
            10,
            5,
            kernel_size=3,
            padding=1,
            padding_mode='circular'
        )

    def forward(self, x):
        # Apply the convolutional layer
        x = self.conv(x)
        
        x = nn.ReLU()(x)
        x = nn.BatchNorm2d(x.shape[1])(x)
        x = torch.sigmoid(x)
        x[:, 0:3, 45:65, 45:65] = 1.0

nca = NCA()
weights = nca.conv.weight.data
print(weights.shape)

state = ti.field(dtype=ti.f32, shape=(w, h, num_com))

@ti.kernel
def conv(state: ti.types.ndarray(), kernel: ti.types.ndarray(), out: ti.types.ndarray()):
    for i, j in state:
        for k in ti.static(range(5)):
            out[i, j] += kernel[k] * state[i + kernel[k][0], j + kernel[k][1]]


torch.Size([5, 10, 3, 3])


In [5]:
x = torch.randn(2, 3)
xa = torch.cat((x, x, x), 0)
xa[0,0] = 10000
x, xa

(tensor([[-0.7078,  0.5516,  1.3847],
         [-1.5035, -0.2386,  0.3490]]),
 tensor([[ 1.0000e+04,  5.5160e-01,  1.3847e+00],
         [-1.5035e+00, -2.3859e-01,  3.4899e-01],
         [-7.0777e-01,  5.5160e-01,  1.3847e+00],
         [-1.5035e+00, -2.3859e-01,  3.4899e-01],
         [-7.0777e-01,  5.5160e-01,  1.3847e+00],
         [-1.5035e+00, -2.3859e-01,  3.4899e-01]]))

In [16]:
c = ti.types.struct(**{
    'a': ti.f32,
    'b': ti.types.vector(n=5, dtype=ti.f32),
    'c': ti.i8})

cfield = c.field(shape=(4,4))
cfield.to_torch()

mem = torch.empty((4,4,7), dtype = torch.f32)


In [4]:

sim = simulation.Simulation(
    'tst', (100,100),
    channels={
        'com': ti.types.vector(n=num_com, dtype=ti.f32),
        'muscles': ti.types.struct(
            flow=ti.types.vector(n=flow_kernel.shape[0], dtype=ti.f32),
            port=ti.f32,
            strength=ti.f32,),
        'macts': ti.types.struct(
            flow=ti.f32,
            port=ti.f32,
            mine=ti.f32),
        'gracts': ti.types.struct(
            flow=ti.types.vector(n=flow_kernel.shape[0], dtype=ti.f32),
            port=ti.f32,
            mine=ti.f32),
        'capital':  {'lims': (0,10)},
        'waste':    {'lims': (0,1)},
        'obstacle': {'lims': (0,1)},
        'port': {
            'lims': (-1,10),
            'metadata': {
                'num_resources': 2,
                'min_regen_amp': 0.5,
                'max_regen_amp': 2,
                'alpha_range': [0.4, 0.9],
                'beta_range': [0.8, 1.2],
                'num_sites_range': [2, 10]}},})

sim.init_data()

In [5]:
d = {'a': 2, 'b': 3}
d[['a', 'b']]

TypeError: unhashable type: 'list'

In [22]:
field = ti.Vector.field(3, ti.i32, shape=(256, 512))
field.shape  # (256, 512)
field.n      # 3

array = field.to_torch().permute(2, 0, 1)
array.shape  # (3, 256, 512)

# field.from_torch(array)  # the input array must be of shape (256, 512, 3)

torch.Size([3, 256, 512])

In [10]:
field = ti.Struct.field({'a': ti.i32, 'b': ti.types.vector(3, float)}, shape=(256, 512))
field.shape # (256, 512)

array_dict = field.to_torch()
array_dict

# array_dict['a'].shape # (256, 512)
# array_dict['b'].shape # (256, 512, 3)

# field.from_numpy(array_dict) # the input array must have the same keys as the field

{'a': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32),
 'b': tensor([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
         